In [8]:
import os
import numpy as np
import random

import warnings
warnings.filterwarnings("ignore")

import jieba
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from argparse import Namespace
import wandb

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_data(train_file, test_file):
    train_data = pd.read_csv(train_file, sep='\t', header=None)
    test_data = pd.read_csv(test_file, sep='\t', header=None)
    train_data[1], lbl = pd.factorize(train_data[1])
    return train_data, test_data, lbl

class BILSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, dropout=0.5):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.hidden2label = nn.Linear(hidden_dim*2, label_size)

    def forward(self, sentence):
        sentence = torch.transpose(sentence, 1, 0)
        x = self.embeddings(sentence)
        lstm_out, self.hidden = self.lstm(x)
        lstm_out = self.dropout(lstm_out)
        y = self.hidden2label(lstm_out[-1, :, :])
        return y

def custom_data_iter(texts, labels):
    for x, y in zip(texts, labels):
        yield x, y

def yield_tokens(data_iter, tokenizer):
    for text, _ in data_iter:
        yield tokenizer(text)

def collate_batch(batch, text_pipeline, device, max_len=20):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        processed_text = F.pad(processed_text, pad=[0, max_len], mode='constant', value=0)
        if len(processed_text) > max_len:
            processed_text = processed_text[:max_len]
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list).T
    return label_list.to(device), text_list.to(device)

def train(dataloader, model, criterion, optimizer, device):
    model.train()
    total_acc, total_count = 0, 0
    total_loss = 0.0
    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        total_loss += loss.item()
    return total_acc / total_count, total_loss / total_count

def evaluate(dataloader, model, criterion, device):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()
    return total_acc / total_count, total_loss / total_count


def predict(dataloader, model, lbl, device):
    model.eval()
    test_pred = []
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text).argmax(1)
            test_pred += list(predicted_label.cpu().numpy())
    return [lbl[x] for x in test_pred]

def run(args):
    wandb.init(project=args.project_name, config=args)
    
    train_data, test_data, lbl = load_data(args.train_file, args.test_file)

    train_iter = custom_data_iter(train_data[0].values[:], train_data[1].values[:])
    tokenizer = jieba.lcut

    vocab = build_vocab_from_iterator(yield_tokens(train_iter, tokenizer), specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])

    def text_pipeline(x): return vocab(tokenizer(x))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    BATCH_SIZE = args.batch_size
    EPOCHS = args.epochs
    LR = args.learning_rate

    num_class = len(lbl)
    vocab_size = len(vocab)
    model = BILSTM(vocab_size, args.embedding_size, 64, num_class).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=args.gamma)
    total_accu = None
    best_accu = 0

    train_iter = custom_data_iter(train_data[0].values[:], train_data[1].values[:])
    train_dataset = to_map_style_dataset(train_iter)
    num_train = int(len(train_dataset) * args.split)
    split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

    train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: collate_batch(x, text_pipeline, device))
    valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: collate_batch(x, text_pipeline, device))

    for epoch in range(1, EPOCHS + 1):
        epoch_start_time = time.time()
        train_acc, train_loss = train(train_dataloader, model, criterion, optimizer, device)
        valid_acc, valid_loss = evaluate(valid_dataloader, model, criterion, device)
        if total_accu is not None and total_accu > valid_acc:
            scheduler.step()
        else:
            total_accu = valid_acc
        if valid_acc > best_accu:
            best_accu = valid_acc
            torch.save(model.state_dict(), args.model_path)
        print('| end of epoch {:3d} | time: {:5.2f}s | train accuracy {:8.3f} | valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, train_acc, valid_acc))
        wandb.log({
            "epoch": epoch,
            "train_accuracy": train_acc,
            "train_loss": train_loss,
            "valid_accuracy": valid_acc,
            "valid_loss": valid_loss
        })

    test_iter = custom_data_iter(test_data[0].values[:], [0] * len(test_data))
    test_dataset = to_map_style_dataset(test_iter)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: collate_batch(x, text_pipeline, device))

    test_pred = predict(test_dataloader, model, lbl, device)
    pd.DataFrame({
        'ID': range(1, len(test_pred) + 1),
        'Target': test_pred,
    }).to_csv('lstm.csv', index=None)
    
    wandb.finish()


seed_everything(42)
args = Namespace(
    data_dir='https://mirror.coggle.club/dataset/coggle-competition/',
    train_file = 'https://mirror.coggle.club/dataset/coggle-competition/intent-classify/train.csv',
    test_file = 'https://mirror.coggle.club/dataset/coggle-competition/intent-classify/test.csv',
    batch_size=32,
    split=0.8,
    embedding_size=100,
    epochs=30,
    learning_rate=1,
    gamma=0.5,
    model_path='BiLSTM.pth',
    project_name='intent-recognition',
)
run(args)

| end of epoch   1 | time:  2.00s | train accuracy    0.139 | valid accuracy    0.248 
| end of epoch   2 | time:  1.99s | train accuracy    0.354 | valid accuracy    0.472 
| end of epoch   3 | time:  1.99s | train accuracy    0.529 | valid accuracy    0.590 
| end of epoch   4 | time:  1.99s | train accuracy    0.639 | valid accuracy    0.662 
| end of epoch   5 | time:  1.99s | train accuracy    0.706 | valid accuracy    0.702 
| end of epoch   6 | time:  2.03s | train accuracy    0.747 | valid accuracy    0.709 
| end of epoch   7 | time:  2.01s | train accuracy    0.784 | valid accuracy    0.760 
| end of epoch   8 | time:  2.01s | train accuracy    0.802 | valid accuracy    0.762 
| end of epoch   9 | time:  2.01s | train accuracy    0.821 | valid accuracy    0.766 
| end of epoch  10 | time:  2.02s | train accuracy    0.843 | valid accuracy    0.769 
| end of epoch  11 | time:  2.00s | train accuracy    0.851 | valid accuracy    0.792 
| end of epoch  12 | time:  2.01s | train a

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_accuracy,▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████
train_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
valid_accuracy,▁▄▅▆▇▇▇▇▇▇████████████████████
valid_loss,█▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃
epoch,30
train_accuracy,0.98347
train_loss,0.00213
valid_accuracy,0.81488
valid_loss,0.03751


In [9]:
from sklearn.metrics import classification_report

def generate_classification_report(model_path, valid_dataloader, model, lbl, device):
    # 加载模型
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for idx, (label, text) in enumerate(valid_dataloader):
            predicted_label = model(text).argmax(1)
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(predicted_label.cpu().numpy())

    # 生成分类报告
    report = classification_report(all_labels, all_preds, target_names=lbl)
    print(report)

# 设置参数
model_path = 'BiLSTM.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
train_data, _, lbl = load_data(args.train_file, args.test_file)
tokenizer = jieba.lcut
vocab = build_vocab_from_iterator(yield_tokens(custom_data_iter(train_data[0].values[:], train_data[1].values[:]), tokenizer), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

def text_pipeline(x): return vocab(tokenizer(x))

train_iter = custom_data_iter(train_data[0].values[:], train_data[1].values[:])
train_dataset = to_map_style_dataset(train_iter)
num_train = int(len(train_dataset) * args.split)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

valid_dataloader = DataLoader(split_valid_, batch_size=args.batch_size, shuffle=True, collate_fn=lambda x: collate_batch(x, text_pipeline, device))

# 定义模型
num_class = len(lbl)
vocab_size = len(vocab)
model = BILSTM(vocab_size, args.embedding_size, 64, num_class)

# 生成分类报告
generate_classification_report(model_path, valid_dataloader, model, lbl, device)

                       precision    recall  f1-score   support

         Travel-Query       0.97      0.96      0.96       243
           Music-Play       0.94      0.95      0.94       273
        FilmTele-Play       0.90      0.95      0.92       262
           Video-Play       0.96      0.95      0.96       264
         Radio-Listen       0.95      0.94      0.95       261
HomeAppliance-Control       0.97      0.97      0.97       239
        Weather-Query       0.95      0.97      0.96       256
         Alarm-Update       0.96      0.97      0.96       238
       Calendar-Query       0.99      0.99      0.99       245
       TVProgram-Play       0.90      0.76      0.82        49
           Audio-Play       0.88      0.84      0.86        43
                Other       0.77      0.57      0.66        47

             accuracy                           0.95      2420
            macro avg       0.93      0.90      0.91      2420
         weighted avg       0.95      0.95      0.95 